# 1. Language Model

- 더 자연스러운 문장에 더 높은 확률을 부여한다.
- GPT (Generative Pre-trained Transformer)
    - 주로 자연어 문장 생성에 특화된 모델
    - 트랜스포머의 디코더 부분만 따로 떼어서 학습 모형으로 사용함.
    - 인코더에서 디코더로의 어텐션은 생략하고, 셀프 어텐션은 순방향만 적용됨.
- 비지도 학습이 가능하다.
- Fine Tuning (미세조정학습)을 통해 지도학습에 활용이 가능하다.

# 2. BERT

- Bidirectional Encoder Representations from Transformers
- 트랜스포머에서 인코더 부분만 사용한 모형, 양방향 셀프 어텐션을 구현하고 있다.
- 트랜스포머의 인코더 부분을 떼어서 사용함.

# 3. Pre-training & Fine-Tuning

## 1) Pre-training

- 언어에 대한 이해를 높이기 위한 비지도학습
- 'masking', 즉 단어를 가리고, 가린 단어를 예측하게 한다.
- Masked language model: 순서와 관계없이 문장 안에서 랜덤한 위치의 단어를 지우고, 모형이 이 단어들을 예측하도록 한다.
- 가려진 단어는 문장의 중간에 위치하며, 양쪽에 단어들이 있어 양방향 셀프 어텐션을 모두 이용해 예측하는 것이 가능하다.
- BERT의 경우, 원래 문장에서 약 15%의 단어들을 마스킹한다.

- 두 개의 문장을 다룰 수 있도록 한다.
- 두 문장을 구분하는 토큰을 정의하고 두 문장 사이에 넣어서 하나의 시퀀스를 만든 후에 인코더에서 한 번에 처리하도록 한다.

## 2) Fine-Tuning

- 실제 수행하고자 하는 작업에 대한 지도학습 (ex: 문서 분류)

# 4. 사전학습된 BERT

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.5 MB/s 
     |████████████████████████████████| 120 kB 70.4 MB/s 
     |████████████████████████████████| 6.6 MB 69.3 MB/s 


- Pipeline
    - 목적에 맞게 적절한 토크나이저와 BERT 모형 객체를 선언한다.
    - 토크나이저를 이용해 주어진 텍스트를 BERT 모형에 맞는 입력으로 변환한다.
    - 변환된 입력을 모형에 전달하고 결과와 확률 등을 받아 전달한다.
    - 토큰 수가 많아지면 (보통 512개) 에러가 발생한다. -> 자동 클래스 사용

- 작업 종류
    - sentiment-analysis : 감성분석
    - text-classification : 문서분류
    - question-answering : 질의응답
    - text-generation : 문서생성
    - translation : 기계번역
    - summarization : 문서요약

In [2]:
from transformers import pipeline

## 감성분석

In [11]:
clf = pipeline('sentiment-analysis')
result = clf('what a beautiful day!')[0]
print('#감성분석 결과: %s, 감성스코어: %0.4f' %(result['label'], result['score']))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


#감성분석 결과: POSITIVE, 감성스코어: 0.9999


## 문서 생성

In [12]:
text_generator = pipeline("text-generation")
result = text_generator("Alice was beginning to get very tired of sitting by her sister on the bank,")
print(result[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Alice was beginning to get very tired of sitting by her sister on the bank, and as she got really carried away, her father had to rush her into the carriage, and by making it harder he would have prevented any more of their doing it."


# 5. 자동 클래스

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [16]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased-finetuned-mrpc')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased-finetuned-mrpc')

input_sentence = 'She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table'
target_sequence = 'She made me angry when she was rude at dinner'

tokens = tokenizer(input_sentence, target_sequence, return_tensors = 'pt')

logits = model(**tokens).logits

results = torch.softmax(logits, dim = 1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f'{label}: {int(round(results[i]*100))}%')

no: 29%
yes: 71%


In [17]:
target_sequence = "The boy quickly ran across the finish line, seizing yet another victory"
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
logits = model(**tokens).logits
results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 95%
yes: 5%


In [18]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [19]:
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  1600
Test set count:  400


In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# cuda를 이용한 GPU 연산이 가능하면 cuda를 사용하고, 아니면 cpu를 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# 모델을 gpu로 옮겨서 연산을 준비
model = model.to(device)

batch_size = 10 # 모형으로 한번에 예측할 데이터의 수
y_pred = [] # 전체 예측결과를 저장

num_batch = len(y_test)//batch_size

for i in range(num_batch):
    inputs = tokenizer(X_test[i*batch_size:(i+1)*batch_size], truncation=True, padding=True, return_tensors="pt")
    # 토큰화 결과를 GPU로 이동
    inputs = inputs.to(device)
    # 모형으로 결과를 예측
    logits = model(**inputs).logits
    # 결과값을 클래스에 대한 확률로 변환
    pred = F.softmax(logits, dim=-1)
    # 예측결과를 CPU로 가져와서 넘파이로 변환한 후, argmax로 확률이 가장 큰 클래스를 선택함
    results = pred.cpu().detach().numpy().argmax(axis=1)
    
    # 전체 예측결과에 추가
    y_pred.extend(results.tolist())

# gpu 메모리를 비움
torch.cuda.empty_cache()

score = sum(y_test == np.array(y_pred))/len(y_test)
print("NLTK 영화리뷰 감성분석 정확도:", score)

NLTK 영화리뷰 감성분석 정확도: 0.8425
